In [30]:
from random import choice
import json
import requests
from bs4 import BeautifulSoup
import urllib2
import ssl
from datetime import datetime

_user_agents = ['Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/65.0.3325.181 Safari/537.36']


class PostScraper:

    def __init__(self, user_agents=None, proxy=None):
        self.user_agents = user_agents
        self.proxy = proxy

    def __random_agent(self):
        if self.user_agents and isinstance(self.user_agents, list):
            return choice(self.user_agents)
        return choice(_user_agents)

    def __request_url(self, url):
        try:
            response = urllib2.urlopen(url)
        except requests.HTTPError:
            raise requests.HTTPError('Received non 200 status code from Instagram')
        except requests.RequestException:
            raise requests.RequestException
        else:
            return response
    
    #Passing downloaded html and parsing it with beautifulsoup to deserialise json 
    @staticmethod
    def extract_json_data(html):
        soup = BeautifulSoup(html, 'html.parser')
        body = soup.find('body')
        script_tag = body.find('script')
        raw_string = script_tag.text.strip().replace('window._sharedData =', '').replace(';', '')
        return json.loads(raw_string)
    
    #Passing a URL of a website to read and collect html using beautifulsoup
    def getpost(self, post_url):
        try:
            response = self.__request_url(post_url)
            html = response.read()
            soup = BeautifulSoup(html, 'html.parser')
        except Exception as e:
            raise e
        else:
            result = soup.prettify('utf-8')
        return result
    


In [31]:
import csv

class IGVariableFinder:

    def __init__(self, user_agents=None, proxy=None):
        self.user_agents = user_agents
        self.proxy = proxy

    def __random_agent(self):
        if self.user_agents and isinstance(self.user_agents, list):
            return choice(self.user_agents)
        return choice(_user_agents)

    def __request_url(self, url):
        try:
            response = urllib2.urlopen(url)
        except requests.HTTPError:
            raise requests.HTTPError('Received non 200 status code from Instagram')
        except requests.RequestException:
            raise requests.RequestException
        else:
            return response
    
    #Get the username of the instagram post from json file
    def findiguser(self, json):
        try:
            user_object = json['entry_data']['PostPage'][0]['graphql']['shortcode_media']['owner']['username']
        except Exception as e:
            raise e
        else: 
            result = str(user_object)
        return result
    
    #Get the time of the instagram post from json file    
    def findigtimestamp(self, json):
        try:
            timestamp = json['entry_data']['PostPage'][0]['graphql']['shortcode_media']['taken_at_timestamp']
            date = datetime.fromtimestamp(timestamp)
        except Exception as e:
            raise e
        else:    
            date_object = date.strftime("%Y/%m/%d")
            time_object = date.strftime("%H:%M:%S")
        return date_object

    #Get the post location of the instagram post from json file    
    def findiglocation(self, json):
        try:
            location_object = json['entry_data']['PostPage'][0]['graphql']['shortcode_media']['location']['slug']
        except:
            result=''
        else: 
            result = str(location_object)
        return result
        
    
    #Get the caption of the instagram post from json file    
    def findigcaption(self, json):
        try:
            raw_caption = json['entry_data']['PostPage'][0]['graphql']['shortcode_media']['edge_media_to_caption']['edges'][0]['node']['text']
            caption_object = raw_caption.encode("ascii", "ignore")
        except Exception as e:
            raise e
        else:
            result = str(caption_object)
        return result

    #Get the first comment of the instagram post from json file - to ensure any hastags missed out from the post caption   
    def findifcomment(self, json):
        try:
            raw_comment = json['entry_data']['PostPage'][0]['graphql']['shortcode_media']['edge_media_to_parent_comment']['edges'][0]['node']['text']
            comment_object = raw_comment.encode("ascii", "ignore")
        except: 
            result = ''
        else: 
            result = str(comment_object)
        return result
        

    #Get the shortcode of the instagram post from json file - part of the link of the post (postID:unique to each post generated by instagram)    
    def findigcode(self, json):
        try:
            shortcode_object = json['entry_data']['PostPage'][0]['graphql']['shortcode_media']['shortcode']
        except Exception as e:
            raise e
        else: 
            result = str(shortcode_object)
        return result

    #Saving each post information into a csv file    
    def savecsv(self, json_file):
        user = self.findiguser(json_file)
        location = self.findiglocation(json_file)
        caption = self.findigcaption(json_file)
        comment = self.findifcomment(json_file)
        publishdate = self.findigtimestamp(json_file)
        shortcode = self.findigcode(json_file)
        
        try:
            with open(r'igpost.csv', mode='a') as igpost_file:
                links_writer = csv.writer(igpost_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
                links_writer.writerow([shortcode, user, location, caption, comment, publishdate])
        except Exception as e:
            raise e
            
            

In [32]:
import pandas as pd

#Class declaration
k = PostScraper()
g = IGVariableFinder()


In [33]:

#Read instagram post link file 
df = pd.read_csv("links.csv", delimiter=",")

#Looping through the list of links and saving information into a csv file 
for index, post_link in df.iterrows():
    results = k.getpost(post_link['IGPostLinks'])
    json_result = k.extract_json_data(results)
    g.savecsv(json_result)
